# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [15]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [16]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [17]:
train_doc = np.random.permutation(open('storage/casestudy_data/train.csv').readlines())
val_doc = np.random.permutation(open('storage/casestudy_data/val.csv').readlines())
batch_size = 8 #experiment with the batch size

### Parameters declaration

Here we are setting up the dimensions we want to use for images. as we know in data there are two types of images 360X360 and 120X160. here we are defining the variables that will be used to define the final dimensions of input images

In [18]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray
nb_classes=5
nb_frames=15

In [19]:
curr_dt_time = datetime.datetime.now()
train_path = 'storage/casestudy_data/train'
val_path = 'storage/casestudy_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =25 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Preprocessing functions
 
 ### functions created to use while preprocessing
 1. init_batch_data: initializes the variables to contain data with finalized dimensions of each frame
 2. get_random_affine: used for affine transformation
 3. normalized_image: used to normalize each channel of each image seprately
 4. crop_and_resize_image: used to crop image and resize it to 120 X 120 dimensions
 5. get_batch_data : used to get data in batch without affine transform and flip(without augmentation)
 6. get_batchdata_augmented: use to get data in batch with augmentation(affine transform and flip image)
 

In [20]:
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [21]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [22]:
def normalize_image(img):
    normalized_image= img - np.percentile(img,15)/ np.percentile(img,85) - np.percentile(img,15)
    return normalized_image

In [23]:
#checked the data and if we crop more than 10 pixels some of images having dimesions 120X160
#are loosing the hand object in image. hence cropping  by 10X10 and then resizing so image wont shrink much too
def crop_and_resize_image(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = imresize(img, (nb_rows,nb_cols))
    return resized_image


In [24]:
def get_batchdata(source_path, folder_list, batch_num, batch_size, t):
    
    batch_data,batch_labels = init_batch_data(batch_size)  
    
    #create a list of image numbers you want to use for a particular video using full frames  
    img_idx = [x for x in range(0, 30,2)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        
        # create a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames for each folder to read them in
        for idx, item in enumerate(img_idx): 
            #image read with simpleITK was giving warnings hence, used cv2
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes 
            resized_image=crop_and_resize_image(image)
            
            #Normal data
            batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0])#normalise and feed in the image
            batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1])#normalise and feed in the image
            batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2])#normalise and feed in the image         

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1       

        
    return batch_data,batch_labels

In [25]:
def get_batchdata_augmented(source_path, folder_list, batch_num, batch_size, t,validation):
    
    # intialize variables to store data read from train data
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # intialize variables for augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    
    # intialize variables for augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = init_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, 30,2)] 
    
    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # create a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames for each folder to read them in
        for idx, item in enumerate(img_idx):             
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes 
            resized_image=crop_and_resize_image(image)
            
            #Normal data
            batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0])#normalise and feed in the image
            batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1])#normalise and feed in the image
            batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2])#normalise and feed in the image
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized_image, M, (resized_image.shape[0], resized_image.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized_image,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

### There are two functions created
1. generator: this function used to get data in batch without augmentation
2. generator_agumentation : this function used to get data with augmentation

### in this final model the generator_augmentation function is used.

In [26]:
def generator(source_path, folder_list, batch_size,ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    if(ablation!=None):
        folder_list=folder_list[:ablation]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield get_batchdata(source_path, folder_list, batch, batch_size, t)
            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield get_batchdata(source_path, folder_list, batch, batch_size, t)

In [27]:
def generator_augmentation(source_path, folder_list, batch_size, validation=False,ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    if(ablation!=None):
        folder_list=folder_list[:ablation]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield get_batchdata_augmented(source_path, folder_list, batch, batch_size, t,validation)
            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield get_batchdata_augmented(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [28]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam


nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

##Conv3D layers
#-----------------------------------------------------------
model.add(Conv3D(nb_featuremap[0], 
                 kernel_size=(5,5,5),
                 input_shape=input_shape,
                 padding='same', name="conv1"))
model.add(Activation('relu'))
#-----------------------------------------------------------
model.add(Conv3D(nb_featuremap[1], 
                 kernel_size=(3,3,3),
                 padding='same',name="conv2"))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#-----------------------------------------------------------
model.add(Conv3D(nb_featuremap[2], 
                 kernel_size=(1,3,3), 
                 padding='same',name="conv3"))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#-----------------------------------------------------------
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling3D(pool_size=(2,2,2)))
#-----------------------------------------------------------

#-----------------------------------------------------------
##Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_dense[1], activation='relu'))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [29]:
optimiser = Adam()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 15, 120, 120, 8)   3008      
_________________________________________________________________
activation_4 (Activation)    (None, 15, 120, 120, 8)   0         
_________________________________________________________________
conv2 (Conv3D)               (None, 15, 120, 120, 16)  3472      
_________________________________________________________________
activation_5 (Activation)    (None, 15, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 7, 60, 60, 16)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 7, 60, 60, 32)     4640      
_________________________________________________________________
activation_6 (Activation)    (None, 7, 60, 60, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

### Choosing DataSize

In [30]:
train_generator = generator_augmentation(train_path, train_doc, batch_size)

val_generator = generator_augmentation(train_path, train_doc, batch_size)

### Model naming and callbacks

In [31]:
model_name = 'Models/Conv_3d_15_epoch25_' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

### Train Model

In [33]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, use_multiprocessing=True,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  storage/casestudy_data/train ; batch size = 8
Source path =  storage/casestudy_data/train ; batch size = 8
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  


83/83 [==============================] - 220s 3s/step - loss: 1.8137 - categorical_accuracy: 0.2761 - val_loss: 1.8141 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to Models/Conv_3d_15_epoch25__2019-03-1406_26_37.718759/model-00001-1.81437-0.27602-1.81407-0.25000.h5
Epoch 2/25
83/83 [==============================] - 129s 2s/step - loss: 1.4061 - categorical_accuracy: 0.3936 - val_loss: 1.4908 - val_categorical_accuracy: 0.3974

Epoch 00002: saving model to Models/Conv_3d_15_epoch25__2019-03-1406_26_37.718759/model-00002-1.40607-0.39357-1.49082-0.39744.h5
Epoch 3/25
83/83 [==============================] - 111s 1s/step - loss: 1.3112 - categorical_accuracy: 0.4478 - val_loss: 1.6194 - val_categorical_accuracy: 0.3878

Epoch 00003: saving model to Models/Conv_3d_15_epoch25__2019-03-1406_26_37.718759/model-00003-1.31882-0.44317-1.61937-0.38782.h5
Epoch 4/25
83/83 [==============================] - 99s 1s/step - loss: 1.2921 - categorical_accuracy: 0.4412 - val_loss: 1.77